# 1D Variability hypothesis testing for HBEC IFN experiment

In [1]:
import scanpy as sc
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
from pybedtools import BedTool
import pickle as pkl
%matplotlib inline

In [2]:
import sys
sys.path.append('/home/ssm-user/Github/scrna-parameter-estimation/dist/memento-0.0.6-py3.8.egg')
sys.path.append('/home/ssm-user/Github/misc-seq/miscseq/')
import encode
import memento

In [3]:
data_path = '/data_volume/memento/hbec/'

### Read the processed RNA data

Focus on the club and bc/club cells and type I interferons for now.

Encode the timestamps to integers.

In [4]:
adata = sc.read(data_path + 'HBEC_type_I_filtered_counts_deep.h5ad')

In [5]:
adata = adata[:, ~adata.var.index.str.startswith('MT-')].copy()
# adata.obs['cell_type'] = adata.obs['cell_type'].apply(lambda x: x if x != 'basal/club' else 'bc')
# adata.obs['cell_type'] = adata.obs['cell_type'].apply(lambda x: x if x != 'ionocyte/tuft' else 'ion-tuft')

/home/ssm-user/anaconda3/envs/single_cell/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


In [6]:
converter = {'basal/club':'BC', 'basal':'B', 'ciliated':'C', 'goblet':'G', 'ionocyte/tuft':'IT', 'neuroendo':'N'}

In [7]:
adata.obs['ct'] = adata.obs['cell_type'].apply(lambda x: converter[x])

### Setup memento

In [8]:
def assign_q(batch):
    
    if batch == 0:
        return 0.387*0.25
    elif batch == 1:
        return 0.392*0.25
    elif batch == 2:
        return 0.436*0.25
    else:
        return 0.417*0.25

In [9]:
adata.obs['q'] = adata.obs['batch'].apply(assign_q)

In [10]:
memento.setup_memento(adata, q_column='q')

Version 0.0.6


### Run memento for each subset, comparing to control

In [11]:
cts = ['C', 'B', 'BC']
tps = ['3', '6', '9', '24', '48']

stims = ['alpha', 'beta', 'gamma', 'lambda']

In [12]:
import os
done_files = os.listdir(data_path + 'binary_test_latest/')

In [13]:
for ct in cts:
    for tp in tps:
        for stim in stims:
            
            fname = '{}_{}_{}.h5ad'.format('-'.join(ct), stim, tp)
            
            if fname in done_files:
                print('Skipping', fname)
                continue

            print('starting', ct, tp, stim)

            adata_stim = adata.copy()[
                adata.obs.ct.isin([ct]) & \
                adata.obs.stim.isin(['control', stim]) & \
                adata.obs.time.isin(['0',tp]), :].copy()
            time_converter={0:0, int(tp):1}
            adata_stim.obs['time_step'] = adata_stim.obs['time'].astype(int).apply(lambda x: time_converter[x])

            memento.create_groups(adata_stim, label_columns=['time_step', 'donor'])
            memento.compute_1d_moments(adata_stim, min_perc_group=.9)

            memento.ht_1d_moments(
                adata_stim, 
                formula_like='1 + time_step + donor',
                treatment_col='time_step', 
                num_boot=10000, 
                verbose=1,
                num_cpus=93,
                resampling='permutation',
                approx=True)

            adata_stim.write(data_path + 'binary_test_latest/{}_{}_{}.h5ad'.format(ct, stim, tp))

starting C 3 alpha


[Parallel(n_jobs=93)]: Using backend LokyBackend with 93 concurrent workers.
[Parallel(n_jobs=93)]: Done  14 tasks      | elapsed:    3.8s
[Parallel(n_jobs=93)]: Done 264 tasks      | elapsed:    5.2s
[Parallel(n_jobs=93)]: Done 614 tasks      | elapsed:    7.4s
[Parallel(n_jobs=93)]: Done 1064 tasks      | elapsed:   10.1s
[Parallel(n_jobs=93)]: Done 1614 tasks      | elapsed:   13.1s
[Parallel(n_jobs=93)]: Done 2264 tasks      | elapsed:   16.7s
[Parallel(n_jobs=93)]: Done 3014 tasks      | elapsed:   20.7s
[Parallel(n_jobs=93)]: Done 3864 tasks      | elapsed:   25.7s
[Parallel(n_jobs=93)]: Done 4814 tasks      | elapsed:   30.8s
[Parallel(n_jobs=93)]: Done 5864 tasks      | elapsed:   36.7s
[Parallel(n_jobs=93)]: Done 7014 tasks      | elapsed:   43.2s
[Parallel(n_jobs=93)]: Done 8264 tasks      | elapsed:   50.0s
[Parallel(n_jobs=93)]: Done 9508 out of 9508 | elapsed:   57.5s finished
... storing 'memento_group' as categorical


starting C 3 beta


/home/ssm-user/anaconda3/envs/single_cell/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
[Parallel(n_jobs=93)]: Using backend LokyBackend with 93 concurrent workers.
[Parallel(n_jobs=93)]: Done  14 tasks      | elapsed:    0.3s
[Parallel(n_jobs=93)]: Done 264 tasks      | elapsed:    1.6s
[Parallel(n_jobs=93)]: Done 614 tasks      | elapsed:    3.5s
[Parallel(n_jobs=93)]: Done 1064 tasks      | elapsed:    5.9s
[Parallel(n_jobs=93)]: Done 1614 tasks      | elapsed:    8.7s
[Parallel(n_jobs=93)]: Done 2264 tasks      | elapsed:   12.0s
[Parallel(n_jobs=93)]: Done 3014 tasks      | elapsed:   15.9s
[Parallel(n_jobs=93)]: Done 3864 tasks      | elapsed:   20.6s
[Parallel(n_jobs=93)]: Done 4814 tasks      | elapsed:   25.4s
[Parallel(n_jobs=93)]: Done 5864 tasks      | elapsed:   31.0s
[Parallel(n_j

starting C 3 gamma


...

/home/ssm-user/anaconda3/envs/single_cell/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
[Parallel(n_jobs=93)]: Using backend LokyBackend with 93 concurrent workers.
[Parallel(n_jobs=93)]: Done  14 tasks      | elapsed:    0.3s
[Parallel(n_jobs=93)]: Done 264 tasks      | elapsed:    1.3s
[Parallel(n_jobs=93)]: Done 614 tasks      | elapsed:    2.9s
[Parallel(n_jobs=93)]: Done 1064 tasks      | elapsed:    4.9s
[Parallel(n_jobs=93)]: Done 1614 tasks      | elapsed:    7.3s
[Parallel(n_jobs=93)]: Done 2264 tasks      | elapsed:   10.3s
[Parallel(n_jobs=93)]: Done 3014 tasks      | elapsed:   13.7s
[Parallel(n_jobs=93)]: Done 3864 tasks      | elapsed:   17.5s
[Parallel(n_jobs=93)]: Done 4814 tasks      | elapsed:   22.0s
[Parallel(n_jobs=93)]: Done 5864 tasks      | elapsed:   26.6s
[Parallel(n_j

starting BC 48 gamma


/home/ssm-user/anaconda3/envs/single_cell/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
[Parallel(n_jobs=93)]: Using backend LokyBackend with 93 concurrent workers.
[Parallel(n_jobs=93)]: Done  14 tasks      | elapsed:    0.3s
[Parallel(n_jobs=93)]: Done 264 tasks      | elapsed:    1.3s
[Parallel(n_jobs=93)]: Done 614 tasks      | elapsed:    2.9s
[Parallel(n_jobs=93)]: Done 1064 tasks      | elapsed:    4.9s
[Parallel(n_jobs=93)]: Done 1614 tasks      | elapsed:    7.2s
[Parallel(n_jobs=93)]: Done 2264 tasks      | elapsed:   10.1s
[Parallel(n_jobs=93)]: Done 3014 tasks      | elapsed:   13.4s
[Parallel(n_jobs=93)]: Done 3864 tasks      | elapsed:   17.2s
[Parallel(n_jobs=93)]: Done 4814 tasks      | elapsed:   21.4s
[Parallel(n_jobs=93)]: Done 5864 tasks      | elapsed:   26.1s
[Parallel(n_j

starting BC 48 lambda


/home/ssm-user/anaconda3/envs/single_cell/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
[Parallel(n_jobs=93)]: Using backend LokyBackend with 93 concurrent workers.
[Parallel(n_jobs=93)]: Done  14 tasks      | elapsed:    0.3s
[Parallel(n_jobs=93)]: Done 264 tasks      | elapsed:    1.3s
[Parallel(n_jobs=93)]: Done 614 tasks      | elapsed:    2.8s
[Parallel(n_jobs=93)]: Done 1064 tasks      | elapsed:    4.8s
[Parallel(n_jobs=93)]: Done 1614 tasks      | elapsed:    7.1s
[Parallel(n_jobs=93)]: Done 2264 tasks      | elapsed:    9.9s
[Parallel(n_jobs=93)]: Done 3014 tasks      | elapsed:   13.2s
[Parallel(n_jobs=93)]: Done 3864 tasks      | elapsed:   16.9s
[Parallel(n_jobs=93)]: Done 4814 tasks      | elapsed:   21.2s
[Parallel(n_jobs=93)]: Done 5864 tasks      | elapsed:   25.8s
[Parallel(n_j